### Imports

In [4]:
import yaml
import pandas as pd
from langchain_openai import ChatOpenAI
from project_info import ProjectInfo
from agentic_interface import AgenticInterface

/Users/elizaknapp/miniconda3/envs/curr_ikigai/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### ENV

In [5]:
ENVIRONMENT = yaml.load(
    open("../project.yaml", "r"), Loader=yaml.FullLoader
)
IKIGAI_API_KEY = ENVIRONMENT["ikigai_api_key"]
OPENAI_API_KEY = ENVIRONMENT["openai_api_key"]

# Setup the llm 
llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0)
json_llm = ChatOpenAI(api_key=OPENAI_API_KEY, temperature=0, model_kwargs={"response_format": {"type": "json_object"}})

### Code

This is a demo of asking 1 query to the agent

In [6]:
# Get all of the project info
user_email = "eliza@ikigailabs.io"
target_project = "Agentic Interface Test"
# target_project = "Demo v2 [Functional]"
mapping_df = mapping = pd.read_csv("./uid_to_name.csv")
project_info = ProjectInfo(
    user_email=user_email,
    api_key=IKIGAI_API_KEY,
    openai_api_key=OPENAI_API_KEY,
    target_project=target_project,
    mapping_df=mapping_df,
    )

In [7]:
agentic_interface = AgenticInterface(llm=llm, json_llm=json_llm, project_info=project_info)
user_query = "How do the average sales in January in the Midwest compare to the average sales in January in the South?"
response = agentic_interface.get_response(user_query)

You asked: How do the average sales in January in the Midwest compare to the average sales in January in the South?
Validating your query...
Query is valid
Finding a dataset to answer your question...
Selected dataset: Sales
Here is the description for your dataset: This dataset contains information about sales transactions of various products in different regions over a period of time. It includes details such as the product name, region, quantity sold, total revenue, total cost, and specific identifiers for each product. The dataset also includes timestamps for each transaction.
Filtering dataset based on your question...
Generating results based on your query...
Finished generating code
Finished running function on data
Finalizing response to your question...
Response finalized


In [8]:
print(response)

The average sales in January in the Midwest were $20,609.71, while the average sales in January in the South were $21,575.49. Therefore, the average sales in January in the South were higher than in the Midwest. This comparison was made using the 'TotalRevenue' column in the dataset, which represents the total revenue generated from sales in January for each region.


In [9]:
# Save to logs
agentic_interface.write_current_state_to_general_logs("./result_files/logs.txt")

Demonstration of asking a follow up question

In [10]:
follow_up_question = "What about in the Northeast also in January?"
# Note that it saves the chat history!
follow_up_response = agentic_interface.get_response(follow_up_question)

You asked: What about in the Northeast also in January?
Validating your query...
First output of prompt validation was not yes or no
Query is valid
Finding a dataset to answer your question...
Selected dataset: Sales
Here is the description for your dataset: This dataset contains information about sales transactions, including details such as the date and time of the transaction, the region where the transaction took place, the product sold, the SKU of the product, the quantity sold, total revenue generated, total cost incurred, and a base quantity. The dataset also includes identifiers for each transaction.
Filtering dataset based on your question...
Generating results based on your query...
Finished generating code
Finished running function on data
Finalizing response to your question...
Response finalized


In [11]:
print(follow_up_response)

The average sales in January in the Northeast were $41,008.14. This information was obtained by calculating the mean of the total revenue generated from sales in January specifically for the Northeast region in the dataset. The 'TotalRevenue' column was used to derive this average sales figure for the Northeast region during the month of January.


Next we will save the logs

In [12]:
# General log saving
agentic_interface.write_current_state_to_general_logs("./result_files/logs.txt")

In [13]:
# Log saving of the entire chat
agentic_interface.upload_chat_history_to_file("./chat_history/demo_chat.csv")